# Training the PQCs

Now all the mandatory ingredients for training a **PQC** for a **CDF** surrogate model are ready. The user can build any training workflow with the functions from **QQuantLib.qml4var** package. 

Present notebook builds this training workflow.

## 1. The Optimizer

Before building the training workflow a mandatory ingredient is an **Optimizer** for updating the trainable parameters of the **PQC** ($\theta$). 

Any **Optimizer** software can be used (the workflows that compute the desired *Loss Function* and their gradients should be provided).

In Deep Learning, one of the most popular **Optimizers** is the *ADAM* one. A minimum working implementation of the *ADAM* is provided in the **QQuantLib.qml4var.adam** module in the function *adam_optimizer_loop*.

The main inputs of the function are:

* weights_dict: dictionary where the keys are the different **PQC** parameter related to the **weitghs** ($\theta$).
* loss_function: properly configurated workflow for computing the desired *Loss function* to optimize.
* metric_function: properly configurated workflow for computing a desired *metric* for monitoring purpouses (not mandatory).
* gradient_function: properly configurated workflow for computing the gradients of the *Loss function* to optimize.
* batch_generator: function for generating batches of the training data.
* initial_time: initial epoch.

In addition to these input a keyword arguments can be provided to the function. For configuring the *ADAM* optimizer the following keywords can be provided:

* learning_rate : learning_rate for ADAM.
* beta1 : beta1 for ADAM.
* beta2 : beta2 for ADAM.

Additionally, the training loop can be configured using the following arguments:

* epochs: maximum number of iterations.
* print_step: print_step for printing evolution of training (the evaluation of the *Loss function* and the *metric* will be printed.
* tolerance: tolerance to achieve. This parameter is used with the *n_counts_tolerance*.
* n_counts_tolerance: number of times the tolerance should be achieved in consecutive iterations.

The *tolerance* and the *n_counts_tolerance* can be used to stop the training loop before the number of desired epochs is achieved. The training will be stopped when the computed tolerance, that is the difference between the *Loss function* after and before a weight updating, be lower than the *tolerance* in *n_counts_tolerance* consecutive steps.

In [ ]:
import sys 
import json
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product

## 2. Example of a training loop.

In [ ]:
dask_client = None

### 2.0 Dask Client 

A *DASK* client can be provided to speed up training.

In [ ]:
from distributed import Client
dask_client = Client()

### 2.1 Get Data

In [ ]:
from benchmark.qml4var.data_sets import create_random_data
cfg_random = {
    "n_points_train": 10, 
    "n_points_test" : 100,
    "minval" : -np.pi,
    "maxval" : np.pi,
    "features_number" : 1
}
x_train, y_train, x_test, y_test = create_random_data(
    **cfg_random
)
plt.plot(x_train, y_train, "o")
plt.plot(x_test, y_test, "-")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Training Dataset", "Testing Dataset"])
plt.title("Random Dataset")

### 2.2 Build PQC

In [ ]:
from QQuantLib.qml4var.architectures import hardware_efficient_ansatz, z_observable, normalize_data

In [ ]:
pqc_cfg = {
    "features_number" : cfg_random["features_number"],
    "n_qubits_by_feature" : 2,
    "n_layers": 3    
}
# Normalization function
base_frecuency, shift_feature = normalize_data(
    [cfg_random["minval"]] * cfg_random["features_number"],
    [cfg_random["maxval"]] * cfg_random["features_number"],
    [-0.5*np.pi] * cfg_random["features_number"],
    [0.5*np.pi] * cfg_random["features_number"]   
)
pqc_cfg.update({
    "base_frecuency" : base_frecuency,
    "shift_feature" : shift_feature    
})   
print(pqc_cfg)
pqc, weights_names, features_names = hardware_efficient_ansatz(**pqc_cfg)
observable = z_observable(**pqc_cfg)

In [ ]:
# plot PQC
circuit = pqc.to_circ()
circuit.display()

### 2.3 QPU info

In [ ]:
from QQuantLib.utils.benchmark_utils import combination_for_list
from QQuantLib.qpu.select_qpu import select_qpu
json_qpu = "../../benchmark/qml4var/JSONs/qpu_ideal.json"
with open(json_qpu) as json_file:
    qpu_dict = json.load(json_file)
qpu_list = combination_for_list(qpu_dict)
qpu_dict = qpu_list[0]
print(qpu_dict)

### 2.4 Configure Loss function

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import qdml_loss_workflow

In [ ]:
# Configuration for workflows
nbshots = 0
# Discretization domain intervals
points = 100
workflow_cfg = {
    "pqc" : pqc,
    "observable" : observable,
    "weights_names" : weights_names,
    "features_names" : features_names,
    "minval" : [cfg_random["minval"]] * cfg_random["features_number"],
    "maxval" : [cfg_random["maxval"]] * cfg_random["features_number"],
    "nbshots" : nbshots,
    "points" : points,
    "qpu_info" : qpu_dict
}

In [ ]:
# How the training function should be computed
training_loss = lambda w_: qdml_loss_workflow(
    w_, x_train, y_train, dask_client=dask_client, **workflow_cfg)

### 2.5 Configure the Loss function gradient computation

In [ ]:
from QQuantLib.qml4var.losses import numeric_gradient

In [ ]:
qdml_loss_workflow_ = lambda w_, x_, y_: qdml_loss_workflow(
    w_, x_, y_, dask_client=dask_client, **workflow_cfg)
numeric_gradient_ = lambda w_, x_, y_: numeric_gradient(
    w_, x_, y_, qdml_loss_workflow_)

### 2.6 Configure a Metric Function

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import mse_workflow

In [ ]:
testing_metric = lambda w_: mse_workflow(
    w_, x_test, y_test, dask_client=dask_client, **workflow_cfg)

### 2.7 Configure a Batch Generator

A batch generator for the data should be built. The main function is splitting the input training data in batches. The updating of the weights will be done after each batch is processed.

The following code implements a very basic batch generator.

In [ ]:
def batch_generator(X, Y, batch_size):
    return [(X[i:i+batch_size] , Y[i:i+batch_size]) for i in range(0, len(X), batch_size)]
batch_size = None
if batch_size is None:
    batch_size = len(x_train)

batch_generator_ = batch_generator(x_train, y_train, batch_size)

## 2.8 Training Time

Now all the ingredients are ready. We need to initialize weights (the *init_weights* function from **QQuantLib.qml4var.architectures** module can be used), configures the optimizer and use the *adam_optimizer_loop* from **QQuantLib.qml4var.adam**

In [ ]:
from QQuantLib.qml4var.architectures import init_weights

In [ ]:
initial_weights = init_weights(weights_names)
print(initial_weights)

The following optimizer configuration will be used. We are going to train for 100 epochs, but if the tolerance decreases below $10^{-5}$ during 10 consecutive epochs the training should be stopped. Additionally, training information will be printed each 10 epochs. 

In [ ]:
# Training configuration
optimizer_cfg = {
    "epochs" : 50,
    "tolerance" : 1.0e-5,
    "print_step" : 10,
    "n_counts_tolerance" : 10
}
# ADAM configuration
optimizer_cfg.update({
    "learning_rate" : 0.1,
    "beta1" : 0.9,
    "beta2" : 0.999,       
})



In [ ]:
from QQuantLib.qml4var.adam import adam_optimizer_loop

In [ ]:
weights_0 = adam_optimizer_loop(
    weights_dict=initial_weights,
    loss_function=training_loss,
    metric_function=testing_metric,
    gradient_function=numeric_gradient_,
    batch_generator=batch_generator_,
    initial_time=0,
    **optimizer_cfg
)

## 3. Results of the Training

We can use the updated weights for evaluate the **PQC** performance

In [ ]:
from QQuantLib.qml4var.myqlm_workflows import workflow_for_cdf, workflow_for_pdf

In [ ]:
cdf_prediction_train = workflow_for_cdf(
    weights_0, x_train, dask_client=dask_client, **workflow_cfg)["y_predict_cdf"]
pdf_prediction_train = workflow_for_pdf(
    weights_0, x_train, dask_client=dask_client, **workflow_cfg)["y_predict_pdf"]
cdf_prediction_test = workflow_for_cdf(
    weights_0, x_test, dask_client=dask_client, **workflow_cfg)["y_predict_cdf"]
pdf_prediction_test = workflow_for_pdf(
    weights_0, x_test, dask_client=dask_client, **workflow_cfg)["y_predict_pdf"]

In [ ]:
plt.plot(x_train, y_train, "o")
plt.plot(x_train, cdf_prediction_train, "o")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Train Data", "Trained PQC"])

In [ ]:
plt.plot(x_test, y_test, "-")
plt.plot(x_test, cdf_prediction_test, "o")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Test Data", "Trained PQC"])

We can easily continue the training by providing the last iteration, updating the *optimizer_cfg* and provided the last obtained weights!

In [ ]:
optimizer_cfg.update({"epochs" : 100})

In [ ]:
weights_0_d = dict(zip(weights_names,weights_0))

In [ ]:
weights_0_d

In [ ]:
weights = adam_optimizer_loop(
    weights_dict=weights_0_d,
    loss_function=training_loss,
    metric_function=testing_metric,
    gradient_function=numeric_gradient_,
    batch_generator=batch_generator_,
    initial_time=50,
    **optimizer_cfg
)

In [ ]:
cdf_prediction_train = workflow_for_cdf(
    weights, x_train, dask_client=dask_client, **workflow_cfg)["y_predict_cdf"]
pdf_prediction_train = workflow_for_pdf(
    weights, x_train, dask_client=dask_client, **workflow_cfg)["y_predict_pdf"]
cdf_prediction_test = workflow_for_cdf(
    weights, x_test, dask_client=dask_client, **workflow_cfg)["y_predict_cdf"]
pdf_prediction_test = workflow_for_pdf(
    weights, x_test, dask_client=dask_client, **workflow_cfg)["y_predict_pdf"]

In [ ]:
plt.plot(x_train, y_train, "o")
plt.plot(x_train, cdf_prediction_train, "o")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Train Data", "Trained PQC"])

In [ ]:
plt.plot(x_test, y_test, "-")
plt.plot(x_test, cdf_prediction_test, "o")
plt.xlabel("Domain")
plt.ylabel("CDF")
plt.legend(["Test Data", "Trained PQC"])